In [1]:
import sklearn
from sklearn.svm import SVC, SVR
from sklearn.model_selection import cross_val_score

from sklearn.metrics import cohen_kappa_score
import numpy as np

In [2]:
import pickle

In [6]:
with open('zorg_gen_aug_nn.pkl', 'rb') as file:
  glv_vec = pickle.load(file)
  y = pickle.load(file)
  new_vec_nn = pickle.load(file)
  new_y_nn = pickle.load(file)
  aug_vec_nn = pickle.load(file)
  aug_y_nn = pickle.load(file)
  aug_v = pickle.load(file)
  aug_y = pickle.load(file)
file.close()

In [7]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

In [8]:
X_train_nn, X_test_nn, y_train_nn, y_test_nn = train_test_split(glv_vec, y, test_size=0.25, random_state=41, stratify=y)


In [9]:
param_grid = {'C': [0.1,1, 10, 100], 'gamma': [0.1,0.1,0.01,0.001],'kernel': ['rbf']}

In [ ]:
xx = X_train_nn.squeeze(1)
cvkf = StratifiedKFold(n_splits = 5, shuffle=True)
rnd_search = RandomizedSearchCV(SVC(), param_grid, cv=cvkf, refit=True, verbose=2)
rnd_search.fit(xx, y_train_nn)
svc = rnd_search.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


[CV] END .......................C=1, gamma=0.001, kernel=rbf; total time=   3.2s
[CV] END .......................C=1, gamma=0.001, kernel=rbf; total time=   3.1s
[CV] END .......................C=1, gamma=0.001, kernel=rbf; total time=   3.2s
[CV] END .......................C=1, gamma=0.001, kernel=rbf; total time=   3.1s
[CV] END .......................C=1, gamma=0.001, kernel=rbf; total time=   3.2s
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time=   4.6s
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time=   4.6s
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time=   4.6s
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time=   4.6s
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time=   4.7s
[CV] END .......................C=10, gamma=0.01, kernel=rbf; total time=   5.2s
[CV] END .......................C=10, gamma=0.01, kernel=rbf; total time=   5.2s
[CV] END ...................

In [ ]:
svc.get_params

<bound method BaseEstimator.get_params of SVC(C=1, gamma=0.001)>

In [ ]:
xx = X_test_nn.squeeze(1)

y_pred = svc.predict(xx)

result_aug = cohen_kappa_score(y_test_nn,y_pred,weights='quadratic')
print("Kappa Score: {}".format(result_aug))

Kappa Score: 0.7120826096825117


In [ ]:
from sklearn.metrics import accuracy_score
_, new_x_test_nn, _, new_y_test_nn = train_test_split(new_vec_nn, new_y_nn, test_size=0.25)
xx = new_x_test_nn.squeeze(1)

y_pred = svc.predict(xx)

result_aug = accuracy_score(new_y_test_nn,y_pred)
print("accuracy Score: {}".format(result_aug))

accuracy Score: 0.33002481389578164


In [ ]:
aug_x_tr, aug_x_ts, aug_y_tr, aug_y_ts = train_test_split(aug_vec_nn, aug_y_nn, test_size=0.2, random_state=41)
xx = aug_x_tr.squeeze(1)
svm = SVC()
rnd_search_nn = RandomizedSearchCV(svm, param_grid, refit=True, verbose=2)
rnd_search_nn.fit(xx, aug_y_tr)
svm = rnd_search_nn.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


[CV] END ........................C=1, gamma=0.01, kernel=rbf; total time=   8.9s
[CV] END ........................C=1, gamma=0.01, kernel=rbf; total time=   8.8s
[CV] END ........................C=1, gamma=0.01, kernel=rbf; total time=   8.8s
[CV] END ........................C=1, gamma=0.01, kernel=rbf; total time=   8.9s
[CV] END ........................C=1, gamma=0.01, kernel=rbf; total time=   9.0s
[CV] END .......................C=10, gamma=0.01, kernel=rbf; total time=  10.4s
[CV] END .......................C=10, gamma=0.01, kernel=rbf; total time=   9.2s
[CV] END .......................C=10, gamma=0.01, kernel=rbf; total time=   9.2s
[CV] END .......................C=10, gamma=0.01, kernel=rbf; total time=  11.1s
[CV] END .......................C=10, gamma=0.01, kernel=rbf; total time=   9.3s
[CV] END ........................C=10, gamma=0.1, kernel=rbf; total time=  10.2s
[CV] END ........................C=10, gamma=0.1, kernel=rbf; total time=  10.1s
[CV] END ...................

In [ ]:
svm.get_params

<bound method BaseEstimator.get_params of SVC(C=10, gamma=0.001)>

In [ ]:
xx = aug_x_ts.squeeze(1)

y_pred = svm.predict(xx)

result_aug = cohen_kappa_score(aug_y_ts,y_pred,weights='quadratic')
print("Kappa Score: {}".format(result_aug))

Kappa Score: 0.6456571791512842


In [10]:
aug_x_tr, aug_x_ts, aug_y_tr, aug_y_ts = train_test_split(aug_v, aug_y, test_size=0.2, random_state=41)
xx = aug_x_tr.squeeze(1)
rnd_search = RandomizedSearchCV(SVC(), param_grid, refit=True, verbose=2)
rnd_search.fit(xx, aug_y_tr)
svm = rnd_search.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END .......................C=100, gamma=0.1, kernel=rbf; total time=  35.6s
[CV] END .......................C=100, gamma=0.1, kernel=rbf; total time=  32.8s
[CV] END .......................C=100, gamma=0.1, kernel=rbf; total time=  32.4s
[CV] END .......................C=100, gamma=0.1, kernel=rbf; total time=  31.8s
[CV] END .......................C=100, gamma=0.1, kernel=rbf; total time=  31.8s
[CV] END .......................C=100, gamma=0.1, kernel=rbf; total time=  31.9s
[CV] END .......................C=100, gamma=0.1, kernel=rbf; total time=  31.9s
[CV] END .......................C=100, gamma=0.1, kernel=rbf; total time=  31.9s
[CV] END .......................C=100, gamma=0.1, kernel=rbf; total time=  31.6s
[CV] END .......................C=100, gamma=0.1, kernel=rbf; total time=  31.8s
[CV] END ........................C=1, gamma=0.01, kernel=rbf; total time=  31.4s
[CV] END ........................C=1, gamma=0.01

In [12]:
svm.get_params

<bound method BaseEstimator.get_params of SVC(C=1, gamma=0.001)>

In [11]:
xx = aug_x_ts.squeeze(1)

y_pred = svm.predict(xx)

result_aug = cohen_kappa_score(aug_y_ts,y_pred,weights='quadratic')
print("Kappa Score: {}".format(result_aug))

Kappa Score: 0.6726539164190903


In [ ]:
xx = X_test_nn.squeeze(1)

y_pred = svm.predict(xx)

result_aug = cohen_kappa_score(y_test_nn,y_pred,weights='quadratic')
print("Kappa Score: {}".format(result_aug))

Kappa Score: 0.8428486879965527
